This notebook generates the figures for Chapter 5: Single-subject statistical analysis from Poldrack, Mumford, and Nichols' _Handbook of fMRI Data Analysis (2nd Edition)_

In [1]:
import os,sys,shutil
from wand.image import Image as WImage
import numpy
import scipy.stats
import matplotlib.pyplot as plt
import nilearn.plotting
%matplotlib inline
import nibabel
from nipype.interfaces import fsl

from nipype.caching import Memory
# use nipype's caching mechanism to save the results of the processing
mem = Memory(base_dir='.')

# set up rpy2 so we can use R magic
%load_ext rpy2.ipython

datadir=os.getenv('FMRIBOOKDATA')
if not datadir:
    raise Exception('You need to set the environment variable FMRIBOOKDATA')
else:
    print 'using base dir:',datadir
sys.path.append('../../utils')

from get_data import get_data
data=get_data()

orig_figuredir='https://web.stanford.edu/group/poldracklab/fmri-handbook-2e-data/figures-1e/'

figuredir=os.path.join(datadir,'../../figures')
if not os.path.exists(figuredir):
    os.mkdir(figuredir)

using base dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data


__Figure 5.1__: Illustration of BOLD timeseries against boxcar

__Figure 5.2__: Features of HRF

__Figure 5.3__: Examples of linear time invariance

__Figure 5.4__: Example of selective averaging

__Figure 5.5__: Example of different basis functions fit to a block design response

__Figure 5.6__: Example of temporal derivative

__Figure 5.7__: Example of FIR model

__Figure 5.8__: Example of FLOBS basis

__Figure 5.9__: Example of fitting data with various basis sets

__Figure 5.10__:  Comparison of convoultion in original time resolution and higher resolution

__Figure 5.11__: Construction of a parametrically modulated regressor

__Figure 5.12__: Example of orthogonalization

__Figure 5.13__: noise structure of fMRI data

__Figure 5.14__: DCT basis set

__Figure 5.15__: Illustration of high-pass filtering

__Figure 5.16__: Illustration of different filtering approaches on blocked designs

__Figure 5.17__: Illustration of different filtering approaches on blocked designs

__Figure 5.18__: How jittering can improve model fit